In [ ]:
import pandas as pd 
import yfinance as yf 
import numpy as np
import math
from math import sqrt
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm as tq
from random import random

In [ ]:
def portfolio_theory(): 
    # Reading in Universe from "Universe Selection.ipynb"
    universe = pd.read_csv("universe.csv")
    
    # 1 Year of Stock Data from Yahoo Finance 
    universe = pd.read_csv("universe.csv")
    hist_data = []
    for stock in tq(universe["Ticker"], desc = "Stock Data"): 
        stock_y = yf.Ticker(stock)
        try: 
            if stock_y.history is not None:
                hist = pd.DataFrame(stock_y.history(period = "1y"))
                df_new = pd.DataFrame()
                df_new[str(stock_y.ticker)] = hist["Close"]
                hist_data.append(df_new)
        except IndexError:
            pass
    
    # Arranging DataFrame 
    hist_data = pd.concat(hist_data, axis = 1)
    
    # Log Percentage Change in stock price 
    hist_ret = hist_data.pct_change().apply(lambda x: np.log(1+x))
    
    # Covariance Matrix
    cov_matrix = hist_ret.cov()

    ## Monté Carlo for Optimal Risky Return Portfolio ## 
    port_ret = []
    port_vol = []
    port_weighting = []

    n_assets = len(universe)
    n_port = 10000

    for port in tq(range(n_port), "Monté Carlo Simulation of Portfolios"):
        # adding random weights within selected universe
        weights = np.random.random(n_assets)
        weights = weights/np.sum(weights)
        port_weighting.append(weights)

        # returns as a function of weighting
        returns = np.dot(weights, individual_er) 
        port_ret.append(returns)

        # Portfolio variance = Weights transposed x (Covariance matrix x Weights)
        var = cov_matrix.mul(weights, axis=0).mul(weights, axis=1).sum().sum()

        # Daily standard deviation
        sd = np.sqrt(var) 

        # 252 trading days in a year on average
        ann_sd = sd*np.sqrt(252) 

        # Annual standard deviation = volatility 
        port_vol.append(ann_sd)
        data = {'Returns':port_ret, 'Volatility':port_vol}

    for port_count, symbol in enumerate(hist_data.columns.tolist()):
        data[symbol+' weight'] = [w[port_count] for w in port_weighting]
    
    # Arranging DataFrame
    df = pd.DataFrame.from_dict(data, orient='index')
    df = df.transpose()
    
    # Adding Sharpes Ratio 
    # Assumed Risk-Free Weight of 1%
    rf = 0.01
    df["Sharpes Ratio"] = ((df['Returns']-rf)/df['Volatility'])
    
    # Minimum Volatility Portfolio
    min_vol_port = df.iloc[df['Volatility'].idxmin()]
    
    # Maximum Return Portfolio
    optimal_risky_port = pd.DataFrame(df.iloc[df["Sharpes Ratio"].idxmax()])
    optimal_risky_port.columns = ["Risky Portfolio Details"]
    
    # Saving Tickers with optimal weighting to csv file
    optimal_risky_port.to_csv(r"optimal portfolio.csv", header = True, sep = "-")
    
    return